In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import subprocess
import re
import itertools
import functools
import collections
import tqdm

In [3]:
import requests

day_num = 21

cookies = {
    'session': '53616c7465645f5f465a94508e1037cf931dedaf44e298b1be500ec71305ff01432a6584b1fe922ea5bab725d96f17b1'
}

url = f'http://adventofcode.com/2017/day/{day_num}/input'

r = requests.get(url, cookies=cookies)

lines = [i.strip() for i in r.text.split('\n')][:-1]

print(f'{len(lines)} lines')

for i in lines[:5]:
    print(i[:10])

108 lines
../.. => #
#./.. => .
##/.. => .
.#/#. => #
##/#. => #


In [6]:
list(itertools.product(range(10), range(3)))

[(0, 0),
 (0, 1),
 (0, 2),
 (1, 0),
 (1, 1),
 (1, 2),
 (2, 0),
 (2, 1),
 (2, 2),
 (3, 0),
 (3, 1),
 (3, 2),
 (4, 0),
 (4, 1),
 (4, 2),
 (5, 0),
 (5, 1),
 (5, 2),
 (6, 0),
 (6, 1),
 (6, 2),
 (7, 0),
 (7, 1),
 (7, 2),
 (8, 0),
 (8, 1),
 (8, 2),
 (9, 0),
 (9, 1),
 (9, 2)]

In [9]:
eye(2) == eye(3)

-c:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


False

In [24]:
def coord_i(a):
    return itertools.product(*map(range, a.shape))

In [23]:
def a_to_num(a):
    t = 0
    for i, j in coord_i(a):
        t <<= 1
        t += a[i, j]
    t += (1 << 20) * len(a)
    return t

In [30]:
def flip(a):
    b = array(a)
    n = len(a)
    for i, j in coord_i(a):
        b[i, n - j - 1] = a[i, j]
    return b

flip(arange(16).reshape((4, 4)))

array([[ 3,  2,  1,  0],
       [ 7,  6,  5,  4],
       [11, 10,  9,  8],
       [15, 14, 13, 12]])

In [29]:
def rotate(a):
    b = array(a)
    n = len(a)
    for i, j in coord_i(a):
        b[i, j] = a[n - 1 - j, i]
    return b

rotate(arange(16).reshape((4, 4)))

array([[12,  8,  4,  0],
       [13,  9,  5,  1],
       [14, 10,  6,  2],
       [15, 11,  7,  3]])

In [33]:
def morphs(a):
    for rotation_count in range(4):
        for flip_count in range(2):
            t = array(a)
            for i in range(flip_count):
                t = flip(t)
            for i in range(rotation_count):
                t = rotate(t)
            yield t

In [44]:
replacements = {}

for i in lines:
    p_in, p_out = re.split(r' => ', i)
    
    p_in = p_in.split('/')
    p_out = p_out.split('/')
    
    n_in = len(p_in)
    n_out = len(p_out)
    
    a_in = zeros((n_in, n_in), dtype=bool)
    a_out = zeros((n_out, n_out), dtype=bool)
    
    for x, y in itertools.product(range(n_in), range(n_in)):
        a_in[x, y] = p_in[x][y] == '#'
        
    for x, y in itertools.product(range(n_out), range(n_out)):
        a_out[x, y] = p_out[x][y] == '#'
        
    for t in morphs(a_in):
        replacements[a_to_num(t)] = a_out

In [40]:
test_lines = """../.# => ##./#../...
.#./..#/### => #..#/..../..../#..#""".split('\n')

replacements = {}

for i in test_lines:
    p_in, p_out = re.split(r' => ', i)
    
    p_in = p_in.split('/')
    p_out = p_out.split('/')
    
    n_in = len(p_in)
    n_out = len(p_out)
    
    a_in = zeros((n_in, n_in), dtype=bool)
    a_out = zeros((n_out, n_out), dtype=bool)
    
    for x, y in itertools.product(range(n_in), range(n_in)):
        a_in[x, y] = p_in[x][y] == '#'
        
    for x, y in itertools.product(range(n_out), range(n_out)):
        a_out[x, y] = p_out[x][y] == '#'
        
    for t in morphs(a_in):
        replacements[a_to_num(t)] = a_out

In [45]:
start = """.#.
..#
###""".split()
start = array([[j == '#' for j in i] for i in start])

In [46]:
def evolve(a):
    n = len(a)
    
    if n == 2 or n == 3:
        return replacements[a_to_num(a)]
    
    if n % 2 == 0:
        m = n // 2
        b = zeros((m * 3, m * 3), dtype=bool)
        for i, j in itertools.product(range(m), range(m)):
            b[i*3:i*3+3,j*3:j*3+3] = replacements[a_to_num(a[i*2:i*2+2,j*2:j*2+2])]
        return b
    
    if n % 3 == 0:
        m = n // 3
        b = zeros((m * 4, m * 4), dtype=bool)
        for i, j in itertools.product(range(m), range(m)):
            b[i*4:i*4+4,j*4:j*4+4] = replacements[a_to_num(a[i*3:i*3+3,j*3:j*3+3])]
        return b

    print(":(")

In [52]:
a = start
for i in tqdm.trange(18):
    a = evolve(a)
a

100%|██████████| 18/18 [00:15<00:00,  1.13it/s] 


array([[ True,  True, False, ...,  True, False,  True],
       [False, False, False, ...,  True,  True,  True],
       [False,  True, False, ..., False,  True, False],
       ..., 
       [ True,  True, False, ..., False, False,  True],
       [False, False, False, ...,  True,  True, False],
       [False,  True, False, ...,  True,  True, False]], dtype=bool)

In [53]:
sum(a)

2480380